In [10]:
import pandas as pd
import requests
import os

df = pd.read_csv('cs552j_A1_dataset_image_id_url.csv')

In [6]:
# Folder where you want to save the image
save_folder = "a1_images"
os.makedirs(save_folder, exist_ok=True)

In [8]:
def download_image(image_url, save_folder):
    image_name = os.path.join(save_folder, image_url.split("/")[-1])
    response = requests.get(image_url)
    if response.status_code == 200:
        with open(image_name, "wb") as file:
            file.write(response.content)
        print(f"Image saved at: {image_name}")
    else:
        print("Failed to download image")

In [12]:
# # downloads all the images to your computer. Do this only once locally, then you can continue.

for i in df.coco_url.values:
    download_image(i, save_folder)

# Your Assessment code below

- please make good use of markdown to separate the sections. 